In [3]:
from bs4 import BeautifulSoup
import logging
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [4]:
link_template = 'https://dblp.org/db/conf/icde/icde2024.html'
def get_title_author(link):
    resp = requests.get(link, proxies=None)
    soup = BeautifulSoup(resp.content, 'html.parser')
    uls = soup.find_all('ul', class_='publ-list')
    titles, authors = [], []
    for ul in uls:  # 遍历所有 ul
        articles = ul.find_all('li', class_='entry inproceedings')  # 找到当前 ul 下的所有 li
        print(f"Found {len(articles)} articles in one ul.")
        for li in articles:
            title = li.find('span', class_='title').text
            author_list = [a.text for a in li.find_all('span', itemprop='name')]

            titles.append(title)
            authors.append(", ".join(author_list[:-1]))

    return titles, authors

In [5]:
titles, authors = get_title_author(link_template)
print(titles, authors)
print(titles[0], authors[0])

Found 501 articles in one ul.
['Evaluating Text-to-SQL Model Failures on Real-World Data.', 'Enabling Roll-Up and Drill-Down Operations in News Exploration with Knowledge Graphs for Due Diligence and Risk Management.', 'Bipartite Graph Analytics: Current Techniques and Future Trends.', 'DeepMapping: Learned Data Mapping for Lossless Compression and Efficient Lookup.', 'PURPLE: Making a Large Language Model a Better SQL Writer.', 'LeaderKV: Improving Read Performance of KV Stores via Learned Index and Decoupled KV Table.', 'TRAP: Tailored Robustness Assessment for Index Advisors via Adversarial Perturbation.', 'Duet: Efficient and Scalable Hybrid Neural Relation Understanding.', 'Knowledge Graph Enhanced Multimodal Transformer for Image-Text Retrieval.', 'Functionality-Aware Database Tuning via Multi-Task Learning.', 'Costream: Learned Cost Models for Operator Placement in Edge-Cloud Environments.', 'S i 1 o F use: Cross-silo Synthetic Data Generation with Latent Tabular Diffusion Model

In [6]:
print(len(titles), len(authors))


501 501


In [7]:
def graph_mask(title):
    t = title.lower()
    conditions = ['graph neural networks', 'gnns', 'gnn', 'graph', 'graph learning', 'graph embedding', 'network embedding']
    return np.any([c in t for c in conditions])

In [8]:
with open(f'2024-ICDE.txt', 'w', encoding='utf8') as fw:
    for t, n in zip(titles, authors):
        if graph_mask(t):
            fw.write(f'1. **{t}**\n\n')
            fw.write(f'    *{n}*\n\n')